# Synchronize Control Rate and Audio Rate

How to synchronize a control rate signal $c[n]$ with a low sampling rate $f_c$ with the audio signal $a[n]$, which has a high sampling rate $f_a$.
The goal is to synchronize the frequency estimates $f_0$ of a pitch tracker to a frequency shifter, which uses this time variable frequency for generating the complex carrier signal.

Upsampling signals for integer upsampling factors L (Oppenheim and Schafer):

* sample rate expansion

* interpolation with a moving average filter (triangle impulse response) or linear interpolation


Miller Puckette in "Theory and Technique of Electronic Music" S.64 about how to convert asynchronous control signals to audio signals:
Three ways to "convert a numeric controlstream to an audio signal":

* As fast as possible: same value like the last control value

* delayed to the nearest sample

* with two-point interpolation for higher delay accuracy

In [ ]:
import numpy as np
import scipy.signal as signal
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from IPython import display as ipd

# CD quality audio sampling rate
fa = 44000
# lower control rate
fc = 4400
# upsampling factor
L = fa / fc
print("The upsampling factor from fc to fa is L:", L)

## Examples:

* _envelope_, which is amplitude modulation

* _vibrato_, which is frequency modulation

In [ ]:
# Envelope in control rate

# length of the signal
dur = 1 # second

# audio signal
t = np.linspace(0, dur, dur*fa)
a = np.sin(2*np.pi*500*t)

# control signal
c = signal.windows.triang(dur*fc)

plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(2)
ax[0].plot(a)
ax[0].set_ylabel("audio")
ax[1].plot(c)
ax[1].set_ylabel("control")
plt.show()
ipd.Audio(a, rate=fa)

These are our two signals.
We want to multiply the control rate envelope $c[n]$ with the audio rate audio signal $c[n]$.
For this purpose, we have to bring $c[n]$ to audio rate.
So we have to fill the spaces between the control signals samples with values that make sense.
There are two ideas to do this for integer upsampling factors $L$:

* _Hold_: Use the original control samples values for the inserted $L-1$ new samples

* _Interpolate_: Insert the new $L-1$ samples in between every control sample with value zero and do linear inperpolation.

First try to hold the control samples value.

In [ ]:
c_hold = np.repeat(c, L)

fig, ax = plt.subplots(2)
ax[0].plot(c)
ax[0].set_ylabel("original")
ax[1].plot(c_hold)
ax[1].set_ylabel("hold")
plt.show()

Now `c_hold` has the right amount of samples.
But as you can see below, the upsampled signal has steps instead of a smooth line.
These are discontinuities which can be audible as clicks or crackling in the audio signal, if the two values are too far apart.

In [ ]:
fig, ax = plt.subplots(2)
ax[0].stem(c[:10])
ax[0].set_ylabel("original")
ax[1].stem(c_hold[:100])
ax[1].set_ylabel("hold")
plt.show()

So let's check the result with this method.

In [ ]:
y_hold = a * c_hold

plt.rcParams['figure.figsize'] = [15, 3]
plt.plot(y_hold)
plt.show()
ipd.Audio(y_hold, rate=fa)

In this case it worked very well because the original control rate was high enough.


## Coarse Amplitude Envelope

Let's assume, we have an amplitude tracker, that outputs one amplitude value for every block of $N$ analyzed audio samples and we want to modulate our signal $a[n]$ with it.
Then $f_c = \frac{f_a}{N}$

In [ ]:
N = 1000
fc = fa / N
print("fc:", fc)

In [ ]:
c = signal.windows.triang(dur*fc)
c_hold = np.repeat(c, N)
y_hold = a * c_hold

plt.plot(y_hold)
plt.show()
ipd.Audio(y_hold, rate=fa)

Now there are audible clicks in the signal


## Frequency Modulation: Vibrato

In [ ]:
# frequency control signal
c = 200 + signal.windows.triang(dur*fc) * 1000
plt.stem(c)
plt.ylabel("f[Hz]")
plt.xlabel("t[samples]")
plt.show()

Now we generate an audio signal $a[n]$ based on the time variable frequency control signal $c[n]$.

In [ ]:
# sampling rate expansion
c_hold = np.repeat(c, N)

a_hold = np.sin(2*np.pi*c_hold*t)

plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(2)
ax[0].plot(c_hold)
ax[1].plot(a_hold)
plt.show()
ipd.Audio(a_hold, rate=fa)

Here the clicks are more present than on the amplitude envelope because the frequency jumps are pretty high.
To smooth this out, we need an expanded control signal with zeros and we might filter it

* with a moving average filter

* or do linear interpolation

In [ ]:
c_zero = np.zeros(dur*fa)
c_zero[::N] = c * 0.5

plt.rcParams['figure.figsize'] = [15, 3]
plt.plot(c_zero)
plt.show()

In [ ]:
# moving average impluse response
h_ma = signal.windows.triang(2*N-1)

# convolve and truncate filter tail
c_zero_ma = signal.convolve(c_zero, h_ma, mode="same")

plt.rcParams['figure.figsize'] = [15, 5]
plt.plot(c_hold)
plt.plot(c_zero_ma)
plt.title("hold vs. zeroing and moving average")
plt.show()

Now generate an audio signal with it.
To do this, we first have to understand how to exactly manipulate the frequency progress of a signal.
This manipulation is frequency modulation (FM).

## Frequency Modulation and Phase Modulation

(see [here](https://de.wikipedia.org/wiki/Frequenzmodulation))

unmodulated carrier: $s(t) = sin(\omega_0 t + p_0)$ with

$\omega_0$ ... Trägerfrequenz

$t$ ... Zeit

$p_0$ ... Phase zum Zeitpunkt $t=0$

momentane Phase: $p(t) = (\omega_0 t + p_0)$

### Phasenmodulation

$s_p(t) = sin(\omega_0 t + p_0 + M_p \cdot m(t))$ mit

$M_p$ ... Modulationsstärke

$m(t)$ ... modulierende Funktion, Modulator

So kann die Phasenmodulation mathematisch recht einfach ausgedrückt werden.

### Frequenzmodulation

Die Frequenzmodulation setzt voraus, dass sich die Frequenz ständig ändert.
Dies lässt sich durch $\omega_0 t$ nicht mehr ausdrücken.
Daher wird die _momentane Kreisfrequenz_

$$\omega(t) = \frac{d}{dt}p(t)$$

eingeführt.
Es ist also die _zeitliche Ableitung der Phasenfunktion_.
Das ist der Kern des Zusammenhangs zwischen Frequenz- und Phasenmodulation.

Eine Frequenzmodulation fordert nun, dass sich die momentane Frequenz nach der folgenden Vorschrift verhält:

$$\omega(t) = \omega_0 + M_f \cdot m(t)$$

Für die Berechnung der Kurvenform zu jedem Zeitpunkt benötigen wir jedoch nicht die momentane Frequenz, sondern die Phasenfunktion:

$$p(t) = \int \omega(t) \, dt$$

Somit ist die Phase für die Frequenzmodulation

$$p_f(t) = \int \omega_0 + M_f m(t) \, dt = \omega_0 \cdot t + p_0 + M_f \int m(t) \, dt$$

und man erhält die Frequenzmodualtion mit dem Ausdruck

$$s_f(t) = sin(\omega_0 t + p_0 + M_f \int m(t) \, dt)$$

Setzen wir nun als Vergleich für PM und FM den Mudulator $m(t) = sin(\omega_m t + p_m)$ ein:

$$s_p(t) = sin(\omega_0 t + p_0 + M_p sin(\omega_m t + p_m))$$

$$s_f(t) = sin(\omega_0 t + p_0 - \frac{M_f}{\omega_m} cos(\omega_m t + p_m))$$

Die momentane Frequenz ist für

* PM: $\omega_p(t) = \omega_0 + M_p \omega_m cos(\omega_m t + p_m)$

* FM: $\omega_f(t) = \omega_0 + M_f sin(\omega_m t + p_m)$ wie gefordert.


Die Modulationsstärke ergibt sich aus

$$M_f = 2 \pi f_\Delta$$

mit $f_\Delta$ ... maximale Frequenzabweichung von $f_0$ (vorausgesetzt $max|m(t)| = 1$)


### Beispiel FM:

Jetzt wollen wir eine Frequenzmodulation machen, um ein Vibrato eines Sinustons zu erzeugen.
Der Sinuston ist der Träger $s_f(t)$ mit einer Grundfrequenz von $f_0$ und der Mudulator $m(t)$ ist ein Sinus mit einer Frequenz von $f_m$.
Die Stärke des Vibratos wird durch die Modulationsstärke $M_f$ bestimmt.

Die Phasen der Schwingungen sind null.
Damit vereinfachen sich die Ausdrücke zu:

$$m(t) = sin(\omega_m t)$$

$$s_f(t) = sin(\omega_0 t + M_f \int m(t) \, dt) = sin(\omega_0 t - \frac{M_f}{\omega_m} cos(\omega_m t))$$

Die Momentankreisfrequenz sollte somit

$$\omega_f = \omega_0 + M_f sin(\omega_m t)$$

sein.

* First we take this formula and calculate the result.

* As a next step, we integrate the modulator numerically, to get a more general code which takes also non sine signals as modulator.


### FM: derived solution for a sine modulator

In [ ]:
dur = 5 # seconds
fs = 44100
t = np.linspace(0, dur, fs*dur)

f0 = 10000
w0 = 2 * np.pi * f0

# modulator
f_delta = 2500
fm = 1
Mf = 2 * np.pi * f_delta
wm = 2 * np.pi * fm

s_f = np.sin(w0 * t - (Mf / wm) * np.cos(wm * t))

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
f_spec, t_spec, Sxx = signal.spectrogram(s_f, fs)
plt.pcolormesh(t_spec, f_spec, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

### FM: processing of an arbitrary modulator function via numeric integration

In [ ]:
# general FM
m = np.sin(wm * t)

# integrate
# this is now: m_int = -(1/wm)*cos(wm*t)
m_int = integrate.cumulative_trapezoid(m, t, initial=0)

s_f_general = np.sin(w0 * t + Mf * m_int)

plt.plot(t, m)
plt.plot(t, m_int)
plt.title("numeric integration")
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
f_spec, t_spec, Sxx = signal.spectrogram(s_f_general, fs)
plt.pcolormesh(t_spec, f_spec, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

So, this is the same result as above.
Now let's try an arbitrary modulator function.

In [ ]:
m = signal.windows.triang(dur*fs)
m_int = integrate.cumulative_trapezoid(m, t, initial=0)

plt.rcParams['figure.figsize'] = [15, 3]
plt.plot(t, m)
plt.plot(t, m_int)
plt.title("numeric integration")
plt.show()

In [ ]:
f_delta = 10000
Mf = 2 * np.pi * f_delta
s_f = np.sin(w0 * t + Mf * m_int)

plt.rcParams['figure.figsize'] = [15, 5]
f_spec, t_spec, Sxx = signal.spectrogram(s_f, fs)
plt.pcolormesh(t_spec, f_spec, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

The important point here is, that we need the integral of the provided frequency function $\omega(t)$ for frequency modulation.
In this manner we can control the frequency of the signal as we want.


## Upsampling for Frequency Modulation

Now let's put this all together to accomplish the task, we want to manage.
The goal is to synchronize the frequency estimates $f_0$ (not to be confused with the carrier frequency $f_0$) of a pitch tracker to a frequency shifter, which uses this time variable frequency for generating the complex carrier signal.
For clarity, we call the frequency estimate signal $c[n]$ because it is the control signal.

So what we want to do is

* generate a control signal $c[n]$ with the low control sampling frequency $f_c$

* upsampling of $c[n]$ with the right method to the audio sampling rate $f_a$ to get $c_{up}[n]$

* FM of a carrier sine signal with the modulator $c_{up}[n]$

In [ ]:
fc = 1
dur = 5
c = np.array((5000, 15000, 7500, 5000, 10000))

fa = 44100
# upsampling factor L
L = fa // fc # just for integer factors
print("L:", L)

# upsampled control signal is the derivative of the modulator for FM
c_up = np.repeat(c, L)

t = np.linspace(0, dur, dur*fa)

plt.plot(c_up)
plt.show()

Since we have the right frequency values in $c[n]$, our carrier base frequency $\omega_0 = 0$.
So the frequency modulation becomes

$$s_f(t) = sin(p_0 + M_f \int m(t) \, dt)$$

In [ ]:
f_delta = 1
Mf = 2 * np.pi * f_delta
m_int = integrate.cumulative_trapezoid(c_up, t, initial=0)

s_f = np.sin(Mf * m_int)

# plot
plt.rcParams['figure.figsize'] = [15, 5]
plt.plot(m_int)
plt.figure()
f_spec, t_spec, Sxx = signal.spectrogram(s_f, fa)
plt.pcolormesh(t_spec, f_spec, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

Do we really need the numerical integration in this case?
If we have a piecewise constant function with values $c[n]$, the integral is always $c[n] \cdot t$.

In [ ]:
f_delta = 1
Mf = 2 * np.pi * f_delta
m_int = c_up * t

s_f = np.sin(Mf * m_int)

# plot
plt.rcParams['figure.figsize'] = [15, 5]
plt.plot(m_int)
plt.figure()
f_spec, t_spec, Sxx = signal.spectrogram(s_f, fa)
plt.pcolormesh(t_spec, f_spec, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

So both methods work, the first is more general for all modulator functions, the second one just works for piecewise constant modulator functions but is computationally cheaper than numerically integrating the whole modulator signal.
The discontinuities in the second method could be a problem.
This might come from the absence of an integration constant

$$\int m_{const}(t) \, dt = c[n] \cdot t + c_0[n]$$